In [11]:
using POMDPs
using GridInterpolations 
#using POMDPModelTools
#using POMDPPolicies
using Parameters
using StaticArrays
using DiscreteValueIteration 

using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using PedestrianAvoidancePOMDP

using FileIO
using JLD2

INFO: Precompiling module PedestrianAvoidancePOMDP.
This may mean module AutomotiveSensors does not support precompilation but is imported by a module that does.
ERROR: LoadError: Declaring __precompile__(false) is not allowed in files that are being precompiled.
Stacktrace:
 [1] _require(::Symbol) at ./loading.jl:455
 [2] require(::Symbol) at ./loading.jl:405
 [3] include_from_node1(::String) at ./loading.jl:576
 [4] include(::String) at ./sysimg.jl:14
 [5] anonymous at ./<missing>:2
while loading /home/xubuntu/stanford/PedestrianAvoidancePOMDP.jl/src/PedestrianAvoidancePOMDP.jl, in expression starting on line 6


LoadError: [91mFailed to precompile PedestrianAvoidancePOMDP to /home/xubuntu/.julia/lib/v0.6/PedestrianAvoidancePOMDP.ji.[39m

In [ ]:
N_PROCS=56
addprocs(N_PROCS)
@everywhere begin 
    using POMDPs
    using GridInterpolations 
    using POMDPModelTools
    using POMDPPolicies
    using Parameters
    using StaticArrays
    using DiscreteValueIteration 

    using AutomotiveDrivingModels
    using AutoViz
    using AutomotiveSensors
    using AutomotivePOMDPs
    using PedestrianAvoidancePOMDP

    pomdp = SingleOCFPOMDP()
end 

solver = ParallelValueIterationSolver(n_procs=N_PROCS, max_iterations=1, belres=1e-4, include_Q=true, verbose=true)



In [ ]:
vi_policy = solve(solver, pomdp)
qmdp_policy = AlphaVectorPolicy(pomdp, vi_policy.qmat, vi_policy.action_map)

# save policy!
save("policy.jld2", "policy", qmdp_policy)


In [ ]:
function AutomotivePOMDPs.action(policy::AlphaVectorPolicy, b::SingleOCFBelief)
    alphas = policy.alphas 
    util = zeros(n_actions(pomdp)) 
    for i=1:n_actions(pomdp)
        res = 0.0
        for (j,s) in enumerate(b.vals)
            si = state_index(pomdp, s)
            res += alphas[i][si]*b.probs[j]
        end
        util[i] = res
    end
    ihi = indmax(util)
    return policy.action_map[ihi]
end


In [10]:
n_states(pomdp)

LoadError: [91mUndefVarError: POMDPPolicies not defined[39m

In [ ]:
pomdp = SingleOCFPOMDP()

qmdp_policy = load("policy.jld")["policy"];


In [ ]:
Pkg.add("Plots")
using Interact
using Plots
gr()

policy_grid = Matrix(length(pomdp.T_RANGE),length(pomdp.S_RANGE))

@manipulate for ego_v in pomdp.EGO_V_RANGE
    @manipulate for ped_v in pomdp.PED_V_RANGE

    #ego_v = 5.38462
    for i=1:length(pomdp.T_RANGE)
        ped_t = pomdp.T_RANGE[i]
        for j=1:length(pomdp.S_RANGE)
            ped_s = pomdp.S_RANGE[j]
            b = SparseCat([SingleOCFState(0.0, ego_v, ped_s, ped_t, 1.57, ped_v)],[1.])
            act = action(qmdp_policy, b) 
          #  println(ped_s, "/", ped_t, " act: ", act.acc)
            policy_grid[i,j] = act.acc
        end
    end

    xs = [pomdp.S_RANGE[i] for i = 1:length(pomdp.S_RANGE)]
    ys = [pomdp.T_RANGE[i] for i = 1:length(pomdp.T_RANGE)]
    heatmap(xs,ys,policy_grid,aspect_ratio=1)


    end
end




In [ ]:
using Interact
using Plots
gr()

policy_grid = Matrix(length(pomdp.T_RANGE),length(pomdp.S_RANGE))

@manipulate for ego_v in pomdp.EGO_V_RANGE
    @manipulate for ped_v in pomdp.PED_V_RANGE

    #ego_v = 5.38462
    for i=1:length(pomdp.T_RANGE)
        ped_t = pomdp.T_RANGE[i]
        for j=1:length(pomdp.S_RANGE)
            ped_s = pomdp.S_RANGE[j]
            b = SparseCat([SingleOCFState(0.0, ego_v, ped_s, ped_t, 1.57, ped_v)],[1.])
            act = action(qmdp_policy, b) 
          #  println(ped_s, "/", ped_t, " act: ", act.acc)
            policy_grid[i,j] = act.lateral_movement
        end
    end

    xs = [pomdp.S_RANGE[i] for i = 1:length(pomdp.S_RANGE)]
    ys = [pomdp.T_RANGE[i] for i = 1:length(pomdp.T_RANGE)]
    heatmap(xs,ys,policy_grid,aspect_ratio=1)


    end
end


In [ ]:
# Test action space


cnt = 0
for (index, a) in enumerate(pomdp.action_space)
    idx = action_index(pomdp,a)
    if (idx != index)
        println("error")
        cnt = cnt + 1
    end
    println(a)
end
println(cnt)


In [ ]:
state_space = states(pomdp)

state_space[state_index(pomdp,SingleOCFState(0.0, 4, 40.0, 0.0, 1.57, 1.5))]

In [ ]:

s = SingleOCFState(0.0, 4.3076923076923075, 40.0, 4.0, 1.57, 1.5)
sp = SingleOCFState(0.0, 4.3076923076923075, 40.0, 4.0, 1.57, 1.5)

act = SingleOCFAction(0.0, 1.0)
reward(pomdp, s, act, sp) 